# **Vehicle Verification - Lab 3 Iris**


## Reference

Iris
- Vehicle Tutorial: https://vehicle-lang.github.io/tutorial/
- Tutorial Repo: https://github.com/vehicle-lang/tutorial
- Iris Dataset: https://en.wikipedia.org/wiki/Iris_flower_data_set

***
# Vehicle - Iris Verification

## Imports

In [1]:
# Local Functions Import
#

# Hack to get local imports from src
import sys
from pathlib import Path

src_folder = Path.cwd().parent.joinpath('src')
if not src_folder.exists():
    raise FileNotFoundError(f'{src_folder} does not exist')
sys.path.insert(0, str(src_folder.resolve()))
del src_folder

# Local Functions
from data import run_setup

In [2]:
# Package imports
#

# General
import os
from datetime import datetime
import time
import sys
import platform
import shutil

# Other
import pandas as pd
import numpy as np
import idx2numpy

# Vehicle specific
import vehicle_lang
from vehicle_lang import Verifier
import onnx, onnxruntime
from maraboupy import Marabou


Instructions for updating:
non-resource variables are not supported in the long term


/opt/anaconda3/envs/Vehicle/lib/python3.11/site-packages/maraboupy/Marabou.py:31: UserWarning: ONNX parser is unavailable because onnx or onnxruntime packages are not installed
  warnings.warn("ONNX parser is unavailable because onnx or onnxruntime packages are not installed")


## Run Setup & Data Load

In [3]:
# Run Setup
#

run_name = 'Vehicle_Iris'
local_project_folder = Path.cwd().parent
data_folder, run_results_folder = run_setup(local_project_folder, run_name)

In [4]:
# Load Iris 'Test' Dataset - As per tutorial
#

iris_test_data_file = data_folder.joinpath('iris_test_data.idx')
iris_test_labels_file = data_folder.joinpath('iris_test_label.idx')

X_test = idx2numpy.convert_from_file(str(iris_test_data_file))
y_test = idx2numpy.convert_from_file(str(iris_test_labels_file))

print(f"Iris Dataset Shape: {X_test.shape}")
print(X_test[:10])
print(f"Iris Dataset Shape: {y_test.shape}")
print(y_test[:10])

# Get original dataset to check names, get value ranges
from sklearn.datasets import load_iris
iris = load_iris()
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_df['species'] = [iris.target_names[label] for label in iris.target]

iris_df.head()

Iris Dataset Shape: (30, 4)
[[6.1 2.8 4.7 1.2]
 [5.7 3.8 1.7 0.3]
 [7.7 2.6 6.9 2.3]
 [6.  2.9 4.5 1.5]
 [6.8 2.8 4.8 1.4]
 [5.4 3.4 1.5 0.4]
 [5.6 2.9 3.6 1.3]
 [6.9 3.1 5.1 2.3]
 [6.2 2.2 4.5 1.5]
 [5.8 2.7 3.9 1.2]]
Iris Dataset Shape: (30, 3)
[[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]]


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


## Verification

In [5]:
# Simple Function to Type Check the Vehicle Script
# return empty list if all ok
#

def vehicle_check(folder, specification):

    specification = data_folder.joinpath(specification)

    result = !vehicle check \
        --specification {specification} \
        
    return result


In [6]:
# Simple Function to Run the Vehicle Script
#

def vehicle_verify(folder, specification, verifier, nn_model, properties):

    specification = folder.joinpath(specification)
    nn_model =  folder.joinpath(nn_model)

    result = !vehicle verify \
        --specification {specification} \
        --verifier {verifier} \
        --network iris:{nn_model} \
        --property {properties}

    return result


In [7]:
# Type Check the spec
#

spec = 'iris.vcl'

check_result = vehicle_check(data_folder, spec)
print('Check OK' if not check_result else '\n'.join(check_result))


Check OK


In [8]:
# Verify Properties
#

spec = 'iris.vcl'
spec_properties = ['property0', 'property1', 'property2', 'property3', 'property4']
verifier = 'Marabou'
models = ['iris_model.onnx']

for spec_property in spec_properties:
    for model in models:
        verification_results = vehicle_verify(data_folder, spec, verifier, model, spec_property)
        print()
        print("=" * 100)
        print(f'Verification Results for: {spec}, {spec_property} with model: {model}')
        print('\n'.join(verification_results))


Verification Results for: iris.vcl, property0 with model: iris_model.onnx


In order to provide support, Vehicle has automatically converted the strict inequalities to non-strict inequalites. This is not sound, but errors will be at most the floating point epsilon used by the verifier, which is usually very small (e.g. 1e-9). However, this may lead to unexpected behaviour (e.g. loss of the law of excluded middle).

See https://github.com/vehicle-lang/vehicle/issues/74 for further details.

Verifying properties:
  property0 [......................................................] 0/1 queries
  property0 [======================================================] 1/1 queries
    result: 🗸 - Marabou proved no counterexample exists

Verification Results for: iris.vcl, property1 with model: iris_model.onnx
Verifying properties:
  property1 [......................................................] 0/8 queries
    result: ✗ - Marabou found a counterexample
      x: [ 4.3, 2.0, 2.951822, 0.1 ]

V